# Julia 機器學習：LIBSVM 支撐向量機

## 作業 029：

請使用 SVM 來建立一個預測模型，來對手寫數字辨識資料集，建立一個分類模型，預測數字為何。

本作業需要使用到的套件有 MLDatasets，請在執行以下範例前先安裝。

```
] add MLDatasets
```

In [4]:
using Pkg
Pkg.add("MLDatasets")

  Updating registry at `C:\Users\Lai\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed DataDeps ──────── v0.7.2
 Installed BufferedStreams ─ v1.0.0
 Installed Blosc ─────────── v0.6.0
 Installed MLDatasets ────── v0.5.2
 Installed HTTP ──────────── v0.8.14
 Installed HDF5 ──────────── v0.12.5
 Installed GZip ──────────── v0.5.1
 Installed MAT ───────────── v0.7.0
 Installed CMakeWrapper ──── v0.2.3
 Installed CMake ─────────── v1.2.0
 Installed IniFile ───────── v0.5.0
  Updating `C:\Users\Lai\.julia\environments\v1.2\Project.toml`
  [eb30cadb] + MLDatasets v0.5.2
  Updating `C:\Users\Lai\.julia\environments\v1.2\Manifest.toml`
  [a74b3585] + Blosc v0.6.0
  [e1450e63] + BufferedStreams v1.0.0
  [631607c0] + CMake v1.2.0
  [d5fb7624] + CMakeWrapper v0.2.3
  [124859b0] + DataDeps v0.7.2
  [92fee26a] + GZip v0.5.1
  [f67ccb44] + HDF5 v0.12.5
  [cd3eb016] + HTTP v0.8.14
  [83e8ac13] + IniFile v0.5.0
  [

In [5]:
using LIBSVM, MLDatasets, Statistics

┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1242


## 讀取資料

呼叫 MNIST 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。

In [6]:
train_X, train_y = MNIST.traindata(1:10000)
test_X,  test_y  = MNIST.testdata(1:1000)

This program has requested access to the data dependency MNIST.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: THE MNIST DATABASE of handwritten digits
Authors: Yann LeCun, Corinna Cortes, Christopher J.C. Burges
Website: http://yann.lecun.com/exdb/mnist/

[LeCun et al., 1998a]
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner.
    "Gradient-based learning applied to document recognition."
    Proceedings of the IEEE, 86(11):2278-2324, November 1998

The files are available for download at the offical
website linked above. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of MNIST aren't really
explicit about any terms of use, so please read the
website to make sure you want to download the
dataset.



Do you want to download the dataset from ["http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyt

(FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

...

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N

## 資料轉換

In [14]:
train_X = reshape(train_X, (10000, 28*28))
test_X = reshape(test_X, (1000, 28*28))

1000×784 reshape(reinterpret(FixedPointNumbers.Normed{UInt8,8}, ::Array{UInt8,3}), 1000, 784) with eltype FixedPointNumbers.Normed{UInt8,8}:
 0.0    0.0  0.0    0.0    0.0    0.0    …  0.0    0.0    0.0    0.0    0.0  
 0.0    0.0  0.671  0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.502
 0.0    0.0  0.996  0.0    0.0    0.408     0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.318  0.0    0.0    0.996     0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.0    0.0    0.62      0.173  0.0    0.0    0.992  0.988
 0.0    0.0  0.0    0.0    0.0    0.0    …  0.898  0.0    0.0    0.984  0.992
 0.0    0.0  0.0    0.282  0.0    0.0       0.992  0.0    0.0    0.984  0.988
 0.0    0.0  0.0    0.682  0.0    0.0       0.494  0.0    0.0    0.122  0.988
 0.0    0.0  0.0    0.984  0.0    0.0       0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.678  0.0    0.0       0.0    0.0    0.0    0.0    0.992
 0.0    0.0  0.0    0.278  0.0    0.0    …  0.0    0.0    0.0    0.0    0.988
 

In [16]:
train_y = Vector{String}(string.(train_y));
test_y = Vector{String}(string.(test_y));

In [17]:
model = LIBSVM.fit!(SVC(), train_X, train_y)

SVC(LIBSVM.Kernel.RadialBasis, 0.0012755102040816326, nothing, 1.0, 3, 0.0, 0.001, true, false, false, LIBSVM.SVM{String}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 784, 10, ["5", "0", "4", "1", "9", "2", "3", "6", "7", "8"], Int32[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], Float64[], Int32[], LIBSVM.SupportVectors{String,FixedPointNumbers.Normed{UInt8,8}}(9999, Int32[863, 1001, 980, 1126, 978, 991, 1032, 1014, 1070, 944], ["5", "5", "5", "5", "5", "5", "5", "5", "5", "5"  …  "8", "8", "8", "8", "8", "8", "8", "8", "8", "8"], FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.498N0f8 … 0.671N0f8 0.0N0f8; … ; 0.816N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8], Int32[1, 12, 36, 48, 66, 101, 133, 139, 146, 174  …  9936, 9937, 9938, 9945, 9949, 9953, 9957, 9971, 9983, 9997], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0),

## 預測

In [18]:
ŷ = String[]
for i = 1:size(test_X, 1)
    y_ = LIBSVM.predict(model, Matrix(test_X[i, :]'))
    append!(ŷ, y_)
end

## 評估

In [21]:
accuracy() = mean((ŷ .== test_y))*100

accuracy (generic function with 1 method)

In [22]:
accuracy()

12.6